In [1]:
import nemo.collections.asr as nemo_asr
# from nemo.collections.asr.models import EncDecMultiTaskModel
# asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_de_conformer_transducer_large")

# asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="nvidia/parakeet-tdt-1.1b")

asr_model = nemo_asr.models.EncDecMultiTaskModel.from_pretrained('nvidia/canary-1b')
# update dcode params
decode_cfg = asr_model.cfg.decoding
decode_cfg.beam.beam_size = 1
asr_model.change_decoding_strategy(decode_cfg)

# asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_de_conformer_ctc_medium")
# 
# asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="stt_multilingual_fastconformer_hybrid_large_pc")


d:\workspace\s2t\.nemo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[NeMo I 2024-05-06 10:42:43 mixins:196] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2024-05-06 10:42:43 mixins:330] Tokenizer SentencePieceTokenizer initialized with 32 tokens
[NeMo I 2024-05-06 10:42:43 mixins:330] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-05-06 10:42:43 mixins:330] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-05-06 10:42:43 mixins:330] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-05-06 10:42:43 mixins:330] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-05-06 10:42:43 aggregate_tokenizer:72] Aggregate vocab size: 4128


[NeMo W 2024-05-06 10:42:43 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    batch_size: null
    num_workers: 8
    use_lhotse: true
    max_duration: 40
    pin_memory: true
    use_bucketing: false
    bucket_duration_bins: null
    num_buckets: 1
    text_field: answer
    lang_field: target_lang
    batch_duration: 360
    quadratic_duration: 15
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2024-05-06 10:42:43 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 

[NeMo I 2024-05-06 10:42:43 features:305] PADDING: 0
[NeMo I 2024-05-06 10:42:49 save_restore_connector:263] Model EncDecMultiTaskModel was successfully restored from C:\Users\vilyi\.cache\huggingface\hub\models--nvidia--canary-1b\snapshots\b9993c06425b28d3d14bdaaf38437153bc4ba30b\canary-1b.nemo.
[NeMo I 2024-05-06 10:42:49 aed_multitask_models:255] Changed decoding strategy to 
    strategy: beam
    compute_hypothesis_token_set: false
    preserve_alignments: null
    compute_langs: false
    beam:
      beam_size: 1
      search_type: default
      len_pen: 1.0
      max_generation_delta: 20
      return_best_hypothesis: true
      preserve_alignments: false
    temperature: 1.0
    


In [2]:
total_params = sum(p.numel() for p in asr_model.parameters())
total_params

1016677408

In [27]:
audiofile = r"D:\Database\s2t-yt\source\xJCfhai9kzc\segments\segment_08.mp3"
transcript = r"D:\Database\s2t-yt\source\xJCfhai9kzc\transcripts\segment_08.txt"    
text = open(transcript, 'r', encoding='utf-8').read().strip()

In [18]:
predicted_text  = asr_model.transcribe([audiofile], 
                                       source_lang='de', 
                                       target_lang='de',
                                       pnc=True)

Transcribing: 1it [00:14, 14.24s/it]


In [28]:
text

'Ich, Ali, hatte vielleicht 4 ausländische Mitschüler in der Klasse. Der Rest war alles Deutsche. Ich hab jetzt mal meinen Neffen abgeholt,'

In [26]:
predicted_text[0]

'Ich , Ali , hatte vielleicht vier ausländische Mitschüler in der Klasse , Der Rest war alles Deutsche . Ich habe mir das noch mal beim Neffen abgeholt .'

In [34]:
from torcheval.metrics import WordErrorRate, BLEUScore, Perplexity

wer = WordErrorRate()
wer.update(text, predicted_text[0])
print(wer.compute())

bleu = BLEUScore(n_gram=4)
bleu.update(text, predicted_text)
print(bleu.compute())

tensor(0.5517)
tensor(0.1846, dtype=torch.float64)
